<a href="https://colab.research.google.com/github/Parthkumar20/VLC-/blob/main/ber_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, Input, regularizers, callbacks
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy import constants, special
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class VLCParameters:
    def __init__(self):
        # Previous LED characteristics with enhanced parameters
        self.led_power_range = (0.5, 2.0)  # Watts
        self.led_wavelength = 450e-9  # meters (Blue LED)
        self.led_bandwidth = 20e6  # Hz
        self.led_beam_angle = np.pi/3  # radians
        self.led_efficiency = 0.8
        self.led_temp_coefficient = -0.003  # Efficiency change per Kelvin

        # Enhanced receiver characteristics
        self.photodetector_area = 1e-4  # m²
        self.photodetector_responsivity = 0.5  # A/W
        self.receiver_fov = np.pi/3  # radians
        self.receiver_gain = 10  # dB
        self.receiver_bandwidth = 100e6  # Hz
        self.detector_quantum_efficiency = 0.8

        # MIMO configuration
        self.mimo_paths = 4
        self.mimo_gain = 10 * np.log10(self.mimo_paths)

        # Enhanced environmental parameters
        self.temperature_range = (300, 400)  # Kelvin
        self.pressure_range = (1e5, 5e6)  # Pascal
        self.distance_range = (1, 50)  # meters
        self.humidity_range = (20, 80)  # percentage
        self.thermal_expansion_coeff = 2.3e-4  # per Kelvin

        # Enhanced noise parameters
        self.thermal_noise_temp = 300  # Kelvin
        self.dark_current = 1e-9  # Ampere
        self.shot_noise_factor = constants.e  # Electron charge
        self.thermal_noise_bandwidth = 1e6  # Hz

        # Enhanced gas properties with wavelength-dependent coefficients
        self.gases = {
            'methane': {
                'absorption_coeff': 0.1,
                'scattering_coeff': 0.05,
                'refractive_index': 1.000444,
                'pressure_broadening': 0.05,
                'temp_dependence': 0.001,
                'molecular_mass': 16.04  # g/mol
            },
            'carbon_dioxide': {
                'absorption_coeff': 0.15,
                'scattering_coeff': 0.07,
                'refractive_index': 1.000449,
                'pressure_broadening': 0.07,
                'temp_dependence': 0.0015,
                'molecular_mass': 44.01
            },
            'hydrogen_sulfide': {
                'absorption_coeff': 0.2,
                'scattering_coeff': 0.1,
                'refractive_index': 1.000644,
                'pressure_broadening': 0.09,
                'temp_dependence': 0.002,
                'molecular_mass': 34.08
            }
        }

        # Enhanced channel parameters
        self.num_reflections = 3
        self.pipe_reflectivity = 0.6
        self.pipe_diameter = 0.3  # meters
        self.pipe_length = 10  # meters

        # Error correction parameters
        self.fec_coding_gain = 5  # dB
        self.rs_code_rate = 0.93  # Reed-Solomon code rate
        self.conv_code_rate = 0.5  # Convolutional code rate
        self.interleaver_depth = 8  # symbols

        # Modulation parameters
        self.modulation_schemes = {
            'QAM256': {'threshold': 30, 'efficiency': 8},
            'QAM64': {'threshold': 25, 'efficiency': 6},
            'QAM16': {'threshold': 20, 'efficiency': 4},
            'QPSK': {'threshold': 15, 'efficiency': 2},
            'BPSK': {'threshold': 10, 'efficiency': 1}
        }

        # OFDM parameters
        self.num_subcarriers = 64
        self.cyclic_prefix_length = 16
        self.pilot_fraction = 0.1

def calculate_temperature_effect(temperature, params):
    """Calculate comprehensive temperature effects on VLC system."""
    # Thermal broadening (Gaussian profile)
    thermal_broadening = np.exp(-(temperature - 300)**2 / (2 * 50**2))

    # Refractive index temperature dependence
    refractive_correction = 1 + params.thermal_expansion_coeff * (temperature - 300)

    # LED efficiency temperature dependence
    led_efficiency = 1 + params.led_temp_coefficient * (temperature - 300)

    # Photodetector temperature effects
    detector_temp_factor = np.exp(-0.01 * (temperature - 300))

    return thermal_broadening * refractive_correction * led_efficiency * detector_temp_factor

def calculate_pressure_effect(pressure, temperature, gas_properties):
    """Calculate comprehensive pressure effects on VLC system."""
    # Pressure broadening (Lorentzian profile)
    gamma = gas_properties['pressure_broadening']
    pressure_broadening = 1 / (1 + ((pressure - 1e5) / (gamma * 1e5))**2)

    # Density-dependent scattering (ideal gas law)
    density_factor = (pressure / 1e5) * (300 / temperature)
    scattering = np.exp(-density_factor * gas_properties['scattering_coeff'])

    # Collision-induced absorption
    collision_factor = (pressure / 1e5)**2 * np.exp(-3000/temperature)

    return pressure_broadening * scattering * (1 - 0.1 * collision_factor)

def custom_loss_function(y_true, y_pred):
    """Enhanced custom loss function with proper tensor handling and logging."""
    # Convert inputs to float32 for numerical stability
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # MSE component with epsilon for numerical stability
    epsilon = 1e-7
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred) + epsilon)

    # Path loss constraint with safe operations
    path_diff = y_pred[1:] - y_pred[:-1]
    path_loss_constraint = tf.reduce_mean(tf.maximum(path_diff, 0.0))

    # Power conservation constraint with safe operations
    power_diff = tf.reduce_mean(tf.abs(y_pred)) - 1.0
    power_constraint = tf.maximum(power_diff, 0.0)

    # Combine all constraints with appropriate weights and logging
    total_loss = (mse_loss +
                  0.01 * path_loss_constraint +
                  0.01 * power_constraint)

    # Add logging for debugging
    tf.debugging.check_numerics(total_loss, 'Loss computation error')

    # Print component values for debugging
    tf.print("\nLoss components:",
             "\nMSE:", mse_loss,
             "\nPath constraint:", path_loss_constraint,
             "\nPower constraint:", power_constraint,
             "\nTotal:", total_loss)

    return total_loss

def generate_enhanced_channel_data(params, num_samples=1000, **kwargs):
    """Generate synthetic VLC channel data with enhanced physical models."""

    def calculate_path_loss(distance, temperature, pressure, gas_type):
        gas_props = params.gases[gas_type]

        # Basic path loss with inverse square law
        basic_loss = 1 / (4 * np.pi * distance**2)

        # Temperature effects
        temp_effect = calculate_temperature_effect(temperature, params)

        # Pressure effects
        pressure_effect = calculate_pressure_effect(pressure, temperature, gas_props)

        # Gas absorption and scattering
        absorption_loss = np.exp(-gas_props['absorption_coeff'] * distance)
        scattering_loss = np.exp(-gas_props['scattering_coeff'] * distance)

        # MIMO diversity gain
        mimo_gain = 10**(params.mimo_gain/10)

        # Beamforming gain (distance-dependent)
        beamforming_gain = 5 * np.exp(-distance/20)

        # Combine all effects
        total_loss = (basic_loss * temp_effect * pressure_effect *
                     absorption_loss * scattering_loss * mimo_gain *
                     beamforming_gain)

        return total_loss

    channels = []
    labels = []

    for _ in range(num_samples):
        distance = kwargs.get('fixed_distance',
                            np.random.uniform(*params.distance_range))
        temperature = kwargs.get('fixed_temperature',
                               np.random.uniform(*params.temperature_range))
        pressure = kwargs.get('fixed_pressure',
                            np.random.uniform(*params.pressure_range))
        gas_type = kwargs.get('fixed_gas',
                             np.random.choice(list(params.gases.keys())))

        # Calculate main channel response
        main_path = calculate_path_loss(distance, temperature, pressure, gas_type)

        # Add MIMO paths
        channel_matrix = np.zeros((params.mimo_paths, params.mimo_paths),
                                dtype=np.complex128)

        for i in range(params.mimo_paths):
            for j in range(params.mimo_paths):
                # Add spatial correlation with geometric considerations
                correlation = np.exp(-abs(i-j)/params.mimo_paths)
                path_variation = np.random.normal(0, 0.1)

                channel_matrix[i, j] = main_path * correlation * (1 + path_variation)

        # Add frequency-selective fading for OFDM subcarriers
        freq_response = np.zeros((params.num_subcarriers, params.mimo_paths,
                                params.mimo_paths), dtype=np.complex128)

        for k in range(params.num_subcarriers):
            freq_selective_fading = np.exp(-k/params.num_subcarriers)
            freq_response[k] = channel_matrix * freq_selective_fading

        # Prepare channel data for neural network
        channel_features = np.stack([np.real(freq_response),
                                   np.imag(freq_response)], axis=-1)

        channels.append(channel_features)
        labels.append([main_path.real, main_path.imag])

    return np.array(channels), np.array(labels)

class DataPreprocessor:
    """Enhanced data preprocessor for VLC channel data with MIMO and OFDM features."""
    def __init__(self):
        self.feature_scaler = StandardScaler()
        self.label_scaler = StandardScaler()
        self.is_fitted = False

    def fit_scalers(self, X, y):
        """Fit all scalers with training data."""
        # Store original shapes for reshaping
        self.X_shape = X.shape

        # Reshape data for fitting scalers
        # Flatten all dimensions except the first (samples)
        X_flattened = X.reshape(X.shape[0], -1)

        # Fit scalers
        self.feature_scaler.fit(X_flattened)
        self.label_scaler.fit(y)

        # Set fitted flag
        self.is_fitted = True

    def preprocess_data(self, X, y, is_training=True):
        """Preprocess the channel data."""
        # Check if we need to fit the scalers
        if is_training and not self.is_fitted:
            self.fit_scalers(X, y)
        elif not self.is_fitted:
            raise ValueError(
                "Preprocessor has not been fitted. Call preprocess_data with "
                "is_training=True first or manually call fit_scalers()."
            )

        # Store shapes for reshaping
        X_shape = X.shape

        # Reshape for preprocessing
        X_flattened = X.reshape(X_shape[0], -1)

        # Transform data using fitted scalers
        X_scaled_flat = self.feature_scaler.transform(X_flattened)
        y_scaled = self.label_scaler.transform(y)

        # Reshape X back to original structure
        X_scaled = X_scaled_flat.reshape(X_shape)

        return X_scaled, y_scaled

    def inverse_transform_labels(self, y):
        """Transform scaled labels back to original scale."""
        if not self.is_fitted:
            raise ValueError("Preprocessor has not been fitted yet.")
        return self.label_scaler.inverse_transform(y)
def create_enhanced_model(input_shape, params):
    """Create enhanced neural network model with improved loss function."""

    # Modified model creation function
    channel_input = Input(shape=input_shape, name='channel_input')

    # Reduce initial layer complexity
    x = layers.Conv3D(32, (3, 3, 3), activation='relu', padding='same',
                     kernel_regularizer=regularizers.l2(0.001))(channel_input)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.1)(x)

    # Simplified MIMO processing branch
    mimo_branch = layers.Conv3D(16, (1, params.mimo_paths, params.mimo_paths),
                              activation='relu', padding='same')(x)

    # Simplified frequency response branch
    freq_branch = layers.Conv3D(16, (params.num_subcarriers, 1, 1),
                              activation='relu', padding='same')(x)

    # Combine branches
    x = layers.Concatenate()([mimo_branch, freq_branch])
    x = layers.Flatten()(x)

    # Reduced complexity in dense layers
    x = layers.Dense(128, activation='relu',
                    kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.Dropout(0.2)(x)

    # Output layer with proper initialization
    outputs = layers.Dense(2, activation='linear',
                         kernel_initializer='he_normal')(x)

    model = models.Model(inputs=channel_input, outputs=outputs)

    # Modified optimizer with reduced learning rate
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=0.0001,  # Reduced learning rate
        clipnorm=1.0,
        amsgrad=True
    )

    # Add custom metrics for monitoring
    metrics = [
        tf.keras.metrics.MeanSquaredError(name='mse'),
        tf.keras.metrics.MeanAbsoluteError(name='mae')
    ]

    model.compile(
        optimizer=optimizer,
        loss=custom_loss_function,
        metrics=metrics
    )

    return model

# Add a custom callback to monitor loss components
class LossComponentCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(f"\nEpoch {epoch + 1} complete")
        if logs is not None:
            for metric_name, metric_value in logs.items():
                print(f"{metric_name}: {metric_value:.6f}")

def create_training_callbacks(model_name='enhanced_vlc_model'):
    """Create enhanced training callbacks with advanced monitoring."""
    return [
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            verbose=1
        ),
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        callbacks.ModelCheckpoint(
            f'{model_name}.keras',
            monitor='val_loss',
            save_best_only=True,
            verbose=1
        ),
        # Add TensorBoard callback for visualization
        callbacks.TensorBoard(
            log_dir='./logs',
            histogram_freq=1,
            write_graph=True,
            update_freq='epoch'
        )
    ]

def calculate_multipath_power(distances, params, num_reflections):
       """Calculate received power considering multipath effects."""
       direct_power = 1 / (4 * np.pi * distances**2)  # Direct path power
       reflected_power = 0
       if num_reflections > 1:
           for i in range(1, num_reflections + 1):
               reflection_loss = params.pipe_reflectivity**i
               reflected_power += reflection_loss / (4 * np.pi * (distances + i * params.pipe_diameter)**2)
       total_power = direct_power + reflected_power
       return 10 * np.log10(total_power)  # Convert to dBm

def generate_enhanced_plots(model, params, history, preprocessor, test_samples=1000):
    """Generate comprehensive analysis plots for enhanced VLC system performance."""
    sns.set_style('whitegrid')
    gases = list(params.gases.keys())

    # Create main figure with updated layout
    fig = plt.figure(figsize=(20, 25))
    gs = plt.GridSpec(5, 2, figure=fig, height_ratios=[1, 1, 1, 1, 1], hspace=0.3, wspace=0.3)
    fig.suptitle('Enhanced VLC System Performance Analysis', fontsize=16, y=0.95)

    # Helper function for theoretical BER calculation with MIMO
    def calculate_theoretical_ber(snr_db, mimo_paths=None): # Add default value for mimo_paths
        snr_linear = 10 ** (snr_db / 10)

        # Use mimo_paths if provided, otherwise assume 1 (no MIMO)
        mimo_gain = mimo_paths if mimo_paths is not None else 1

        effective_snr = snr_linear * mimo_gain
        return 0.5 * special.erfc(np.sqrt(effective_snr) / np.sqrt(2))

    # 1. Enhanced BER vs SNR plot
    ax1 = fig.add_subplot(gs[0, 0])
    snr_range = np.linspace(-10, 30, 100)
    colors = plt.cm.rainbow(np.linspace(0, 1, len(gases)))

    for gas, color in zip(gases, colors):
        bers = []
        for snr in snr_range:
            # Include MIMO and coding gains
            effective_snr = snr + params.mimo_gain + params.fec_coding_gain
            ber = calculate_theoretical_ber(effective_snr, params.mimo_paths)
            bers.append(ber)
        ax1.semilogy(snr_range, bers, label=f'{gas} (MIMO)', color=color)

    ax1.set_xlabel('SNR (dB)')
    ax1.set_ylabel('Bit Error Rate')
    ax1.set_title('Enhanced BER vs SNR with MIMO and FEC')
    ax1.grid(True)
    ax1.legend()

        # 2. BER vs Distance for different gases
    ax2 = fig.add_subplot(gs[0, 1])
    distances = np.linspace(*params.distance_range, 20)

    for gas, color in zip(gases, colors):
        bers = []
        for distance in distances:
            X_test, y_test = generate_enhanced_channel_data(params, num_samples=100,
                                                 fixed_gas=gas, fixed_distance=distance)
            X_test_scaled, _ = preprocessor.preprocess_data(X_test, y_test, is_training=False)
            predictions = model.predict(X_test_scaled, verbose=0)
            ber = calculate_theoretical_ber(20 - 20*np.log10(distance))
            bers.append(ber)
        ax2.semilogy(distances, bers, label=gas, color=color)

    ax2.set_xlabel('Distance (m)')
    ax2.set_ylabel('Bit Error Rate')
    ax2.set_title('BER vs Distance for Different Gases')
    ax2.grid(True)
    ax2.legend()

    # 3. Channel Capacity Analysis
    ax3 = fig.add_subplot(gs[1, 0])
    distances = np.linspace(*params.distance_range, 100)

    for gas, color in zip(gases, colors):
        gas_props = params.gases[gas]
        capacities = []
        for d in distances:
            path_loss = 1 / (4 * np.pi * d**2)
            attenuation = np.exp(-gas_props['absorption_coeff'] * d)
            snr = 20 * np.log10(path_loss * attenuation)
            capacity = params.led_bandwidth * np.log2(1 + 10**(snr/10)) / 1e6
            capacities.append(capacity)
        ax3.plot(distances, capacities, label=gas, color=color)

    ax3.set_xlabel('Distance (m)')
    ax3.set_ylabel('Channel Capacity (Mbps)')
    ax3.set_title('Channel Capacity vs Distance for Different Gases')
    ax3.grid(True)
    ax3.legend()

    # 4. Path Loss Analysis
    ax4 = fig.add_subplot(gs[1, 1])

    for gas, color in zip(gases, colors):
        gas_props = params.gases[gas]
        path_losses = []
        for d in distances:
            basic_loss = 20 * np.log10(4 * np.pi * d / params.led_wavelength)
            gas_loss = 10 * np.log10(np.exp(-gas_props['absorption_coeff'] * d))
            path_losses.append(basic_loss + gas_loss)
        ax4.plot(distances, path_losses, label=gas, color=color)

    ax4.set_xlabel('Distance (m)')
    ax4.set_ylabel('Path Loss (dB)')
    ax4.set_title('Path Loss vs Distance for Different Gases')
    ax4.grid(True)
    ax4.legend()

    # 5. Temperature Effect on BER
    ax5 = fig.add_subplot(gs[2, 0])
    temperatures = np.linspace(*params.temperature_range, 20)

    for gas, color in zip(gases, colors):
        bers = []
        for temp in temperatures:
            X_test, y_test = generate_enhanced_channel_data(params, num_samples=100,
                                                 fixed_gas=gas, fixed_temperature=temp)
            X_test_scaled, _ = preprocessor.preprocess_data(X_test, y_test, is_training=False)
            predictions = model.predict(X_test_scaled, verbose=0)
            temp_factor = np.exp(0.01 * (temp - 300))
            ber = calculate_theoretical_ber(15) * temp_factor
            bers.append(ber)
        ax5.semilogy(temperatures, bers, label=gas, color=color)

    ax5.set_xlabel('Temperature (K)')
    ax5.set_ylabel('Bit Error Rate')
    ax5.set_title('Temperature Effect on BER')
    ax5.grid(True)
    ax5.legend()

    # 6. Pressure Effect on BER
    ax6 = fig.add_subplot(gs[2, 1])
    pressures = np.linspace(*params.pressure_range, 20)

    for gas, color in zip(gases, colors):
        bers = []
        for pressure in pressures:
            X_test, y_test = generate_enhanced_channel_data(params, num_samples=100,
                                                 fixed_gas=gas, fixed_pressure=pressure)
            X_test_scaled, _ = preprocessor.preprocess_data(X_test, y_test, is_training=False)
            predictions = model.predict(X_test_scaled, verbose=0)
            pressure_factor = np.exp(0.1 * (pressure - params.pressure_range[0]) / 1e5)
            ber = calculate_theoretical_ber(15) * pressure_factor
            bers.append(ber)
        ax6.semilogy(pressures/1e5, bers, label=gas, color=color)

    ax6.set_xlabel('Pressure (bar)')
    ax6.set_ylabel('Bit Error Rate')
    ax6.set_title('Pressure Effect on BER')
    ax6.grid(True)
    ax6.legend()

    # 7. Multipath Effects
    ax7 = fig.add_subplot(gs[3, 0])

    for gas, color in zip(gases, colors):
        gas_props = params.gases[gas]
        direct_power = calculate_multipath_power(distances, params, 1)
        reflected_power = calculate_multipath_power(distances, params, params.num_reflections)
        attenuation = np.exp(-gas_props['absorption_coeff'] * distances)
        total_power = (direct_power + reflected_power) * attenuation
        ax7.plot(distances, total_power, label=gas, color=color)

    ax7.set_xlabel('Distance (m)')
    ax7.set_ylabel('Received Power (dBm)')
    ax7.set_title('Multipath Effects Analysis')
    ax7.grid(True)
    ax7.legend()

    # 8. System Reliability
    ax8 = fig.add_subplot(gs[3, 1])

    for gas, color in zip(gases, colors):
        success_rates = []
        for d in distances:
            X_test, y_test = generate_enhanced_channel_data(params, num_samples=100,
                                                 fixed_gas=gas, fixed_distance=d)
            X_test_scaled, _ = preprocessor.preprocess_data(X_test, y_test, is_training=False)
            predictions = model.predict(X_test_scaled, verbose=0)
            ber = calculate_theoretical_ber(20 - 20*np.log10(d))
            success_rate = np.mean(ber < params.target_ber)
            success_rates.append(success_rate)
        ax8.plot(distances, success_rates, label=gas, color=color)

    ax8.set_xlabel('Distance (m)')
    ax8.set_ylabel('Success Rate')
    ax8.set_title('System Reliability Analysis')
    ax8.grid(True)
    ax8.legend()

    return fig


def main():
    """Main execution function that ties together all components of the VLC system."""
    try:
        # Step 1: Initialize system parameters
        params = VLCParameters()

        # Step 2: Create data preprocessor
        preprocessor = DataPreprocessor()

        # Step 3: Generate synthetic channel data
        print("Generating channel data...")
        X_data, y_data = generate_enhanced_channel_data(params, num_samples=10000)

        # Step 4: Split data into training, validation, and test sets
        print("Splitting data into train/validation/test sets...")
        X_train, X_temp, y_train, y_temp = train_test_split(
            X_data, y_data,
            test_size=0.3,
            random_state=42
        )

        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp,
            test_size=0.5,
            random_state=42
        )

        # Step 5: Preprocess data
        print("Preprocessing data...")
        X_train_scaled, y_train_scaled = preprocessor.preprocess_data(
            X_train, y_train,
            is_training=True
        )

        X_val_scaled, y_val_scaled = preprocessor.preprocess_data(
            X_val, y_val,
            is_training=False
        )

        X_test_scaled, y_test_scaled = preprocessor.preprocess_data(
            X_test, y_test,
            is_training=False
        )

        # Step 6: Create neural network model
        print("Creating neural network model...")
        model = create_enhanced_model(X_train_scaled.shape[1:], params)

        # Step 7: Set up training callbacks
        print("Setting up training callbacks...")
        callbacks_list = create_training_callbacks()

        # Step 8: Train model
        print("Beginning model training...")
        history = model.fit(
            X_train_scaled, y_train_scaled,
            validation_data=(X_val_scaled, y_val_scaled),
            epochs=100,
            batch_size=32,
            callbacks=callbacks_list,
            verbose=1
        )

        # Step 9: Generate analysis plots
        print("Generating analysis plots...")
        fig = generate_enhanced_plots(
            model,
            params,
            history,
            preprocessor,
            test_samples=1000
        )
        plt.show()

        # Step 10: Evaluate model performance
        print("\nEvaluating model performance...")
        test_loss = model.evaluate(
            X_test_scaled,
            y_test_scaled,
            verbose=1
        )

        print("\nFinal Test Results:")
        print(f"Loss: {test_loss:.4f}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Stack trace:")
        import traceback
        traceback.print_exc()
        raise

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
128/219 ━━━━━━━━━━━━━━━━━━━━ 1:22 904ms/step - loss: 0.0336 - mae: 0.0079 - mse: 0.0297
Loss components: 
MSE: 2.92036475e-05 
Path constraint: 0.00119959249 
Power constraint: 0 
Total: 4.11995716e-05
129/219 ━━━━━━━━━━━━━━━━━━━━ 1:21 904ms/step - loss: 0.0336 - mae: 0.0079 - mse: 0.0297
Loss components: 
MSE: 2.89089312e-05 
Path constraint: 0.00140652701 
Power constraint: 0 
Total: 4.29742e-05
130/219 ━━━━━━━━━━━━━━━━━━━━ 1:20 903ms/step - loss: 0.0336 - mae: 0.0079 - mse: 0.0298
Loss components: 
MSE: 1.80445204e-05 
Path constraint: 0.000978989061 
Power constraint: 0 
Total: 2.78344105e-05
131/219 ━━━━━━━━━━━━━━━━━━━━ 1:19 902ms/step - loss: 0.0337 - mae: 0.0079 - mse: 0.0298
Loss components: 
MSE: 3.17791273e-05 
Path constraint: 0.00164094474 
Power constraint: 0 
Total: 4.81885727e-05
132/219 ━━━━━━━━━━━━━━━━━━━━ 1:18 902ms/step - loss: 0.0337 - mae: 0.0079 - mse: 0.0298
Loss components: 
MSE: 2.56763378e-05 
Path constraint: